# Victor Augusto de Lima Cruz 2020054854

Apresentação: https://youtu.be/HdYmt4l_-uc

# Reprodução
Para reproduzir o trabalho como um todo é necessário ter as pastas no diretório atual:

*   data_apple
*   data_google
*   data_samsung

Também são necessárias as dependências a seguir:

*   sklearn
*   pdfminer.six
*   pandas
*   numpy
*   transformers
*   torch
*   langchain
*   nltk

Por último é necessário ter um token para se utilizar o modelo llama2 escolhido


# Tarefas

A tarefa propsta para o trabalho final consiste em analisar e comparar relatórios ambientais de produtos tecnológicos.
Para isso:


*   Serão coletados os relatórios em PDFs disponíveis nos sites das empresas.
*   Suas informações serão extraídas para txts.
*   Algum LLM será usado para sumarizar/estruturar o texto de acordo com algumas categorias.
*   Os resultados serão analisados e será feita a comparação dos relatórios.





#Imports

Importando as bibliotecas e funções necessárias.

In [ ]:
import os
import re
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from pdfminer.high_level import extract_text
import transformers
import torch
from langchain.prompts import PromptTemplate
from nltk.corpus import stopwords
import nltk
from nltk.tokenize import word_tokenize

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')

# Dados utilizados

Relatórios de sustentabilidade de celulares da Apple, Google e Samsung.
Os dados estão disponíveis em:



*   https://www.apple.com/environment/
*   https://sustainability.google/reports/
*   https://www.samsung.com/global/sustainability/focus/products/mobile/




Os dados tanto de origem quanto os que são gerados ao longo da execução podem ser pedidos por email 

# Extraindo texto dos PDFs

O primeiro passo para a tarefa realizada foi extrair os textos dos PDFs para txt de forma que possam ser usados em seguida.
Para isso foi utilizada a biblioteca pdfminer.six. Ela possui parâmetros para ajustar a extração do texto, mas os defaults conseguiram ter um bom resultado na extração portanto foram os utilizados.

Os únicos erros percebidos foram em tabelas presente nos pdfs em que os números extraídos ficam na ordem errada. Mas por ainda estarem juntos não deve afetar o resultado da tarefa proposta.

Os textos são extraídos da pasta data_empresa para a pasta extracted_empresa

Caso os pdf sejam muitos grandes apenas os primeiros 10000 caracteres foram usados na extração, devido à limites computacionais do modelo usado a seguir.

In [ ]:
def extractTextFromPdf(pdfPath):
    with open(pdfPath, 'rb') as file:
        text = extract_text(file)
    return text

def extractPdfsToTxt(pdfsDirectoryPath, extractionDirectoryPath):
  filesnames = os.listdir(pdfsDirectoryPath)
  for files in filesnames:
    if files.split('.')[1] != 'pdf':
      continue
    print(files.split('.')[0] + '.txt')
    pdfPath = pdfsDirectoryPath + '/' + files
    extracted_text = extractTextFromPdf(pdfPath)
    with open(extractionDirectoryPath + '/' + files.split('.')[0] + '.txt', 'w') as file:
      file.write(extracted_text[:10000])

In [ ]:
extractPdfsToTxt('data_google', 'extracted_google')

In [ ]:
extractPdfsToTxt('data_apple', 'extracted_apple')

In [ ]:
extractPdfsToTxt('data_samsung', 'extracted_samsung')

#Utilizando o Llama2

Para esse trabalho foram testados os modelos Llama2 (7B) e Mistral (7B), com os prompts testados o llama2 obteve um melhor resultado, sendo assim escolhido para o trabalho.

Foram testados diversos prompts para tentar chegar ao resultado desejado e optou-se por criar um sumário baseado em tópicos. Isso permite uma melhor estruturação das informações facilitando possíveis comparações entre os produtos. Um padrão que foi notado nos testes é que o llama se desempenha melhor e mais consistentemente quando a descrição da tarefa está no final do texto.

Dessa forma o prompt final foi:

    Environmental Report

    Categories:
    1. Energy Efficiency
    2. Recycled and Renewable Content
    3. Longevity and Durability
    4. Emission Reduction Initiatives
    5. Material Use
    6. Packaging
    7. Ethical Sourcing and Supplier Code of Conduct

    Using the information from the text, write a detailed summary for each of the seven categories above. Be sure to include specific data and statistics where available.

Os prompts são criados usando o template acima com o Environmental Report sendo substituido pelos textos dos arquivos no diretório extracted. Os textos gerados são salvos no diretório generated_text

Para executar as queries foram usadas as bibliotecas LangChain e transformers que permitem importar e utilizar o modelo. O modelo acima consome bastante memória RAM e é interessante que se tenha uma GPU pois somente com CPU os textos um tempo considerável para serem gerados.

O modelo exige que se tenha uma chave de API para que possa ser utilizado.


In [ ]:
def runPromptsAndSaveResultsInDirectory(inputDirectoryPath, outputDirectoryPath):
    model = "meta-llama/Llama-2-7b-chat-hf"
    tokenizer = transformers.AutoTokenizer.from_pretrained(model)
    pipeline = transformers.pipeline(
        "text-generation",
        model=model,
        torch_dtype=torch.float16,
        device_map="auto",
    )

    template = """
    {environmentalReport}

    Categories:
    1. Energy Efficiency
    2. Recycled and Renewable Content
    3. Longevity and Durability
    4. Emission Reduction Initiatives
    5. Material Use
    6. Packaging
    7. Ethical Sourcing and Supplier Code of Conduct

    Using the information from the text, write a detailed summary for each of the seven categories above. Be sure to include specific data and statistics where available.

    """

    prompt = PromptTemplate(input_variables=["environmentalReport"], template=template)


    files = os.listdir(inputDirectoryPath)

    for filename in files:
        filepath = inputDirectoryPath + "/" + filename
        if(filepath.split('.')[1] != 'txt'):
            continue
        print('filepath - ',filepath)

        environmentalReport =  None
        with open(filepath, 'r') as file:
            environmentalReport = file.read()

        environmentalReportPrompt = prompt.format(environmentalReport=environmentalReport)

        sequences = pipeline(
            environmentalReportPrompt,
            do_sample=True,
            top_k=10,
            num_return_sequences=1,
            eos_token_id=tokenizer.eos_token_id
        )

        ending = "Using the information from the text, write a detailed summary for each of the seven categories above. Be sure to include specific data and statistics where available."

        for sequence in sequences:
            if ending in sequence['generated_text']:
                result = (sequence['generated_text'].split(ending))[1]
                writefilename = outputDirectoryPath + '/' + filename.split('.')[0] + '-res.txt'
                print(result)
                print(writefilename)
                with open(writefilename, 'w') as file:
                    file.write(result)

In [ ]:
runPromptsAndSaveResultsInDirectory('extracted_google','generated_text_google')

In [ ]:
runPromptsAndSaveResultsInDirectory('extracted_apple','generated_text_apple')

In [ ]:
runPromptsAndSaveResultsInDirectory('extracted_samsung','generated_text_samsung')

# Análise dos prompts

Com todos os textos gerados que fazem um sumário estruturado dos dados dos celulares, optou-se por estruturar esses resultados em um dataframe.

Foi feito um parse do texto gerado, a estrutura de tópicos quase sempre aparece ele, nem sempre da mesma forma. Alguns padões observados foram:

      Emission Reduction Initiatives:
      Google has implemented various initiatives to reduce...

      Material Use:
      The Pixel 2 is designed to use as few...

ou
      
      1. Energy Efficiency: The Pixel 8...

      2. Recycled and Renewable Content: The Pixel 8 is...
ou

      1. Energy Efficiency
    
      Samsung is committed to reducing energy consumption...

      2. Recycled and Renewable Content
    
      Samsung is committed to increasing the use of recycled...



O parsing foi feito de forma que a informação fosse extraída independentemente de como foi gerada, desde de que a estrutura de tópicos tenha sido gerada corretamente.

Um texto interessante que dá a ideia que os resultados são coerentes é:

      The report does not provide specific data on the longevity and durability of the iPhone 4S.

## Auxiliares para análise dos resultados

In [ ]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', '{:.3f}'.format)

In [ ]:
def parseGeneratedTextCategories(generatedTextDirectory,categories):
  files = os.listdir(generatedTextDirectory)
  result = {}
  for filename in files:
    if filename.split('.')[1] != 'txt':
      continue
    result[filename] = {}

    pattern = re.compile(r'(\d\d\d\d\d\d)')
    match = pattern.search(filename)
    if match:
      date = match.group()
      year = date[4:]
      month = date[:2]
      result[filename]['month'] = month
      result[filename]['year'] = year

    content = None
    with open(generatedTextDirectory + '/' + filename) as f:
      content = f.read()
    content = content.replace('\n\n', '|')

    for category in categories:
      content = re.sub((r'(\s*\d*\.?\s*' + category + r':?)\|'), r'\1 ', content)

    line_split = content.split('|')
    for text in line_split:
      acum = ''
      cur_cat = ''
      for text2 in text.split('\n'):
        for category in categories:
          if(category in text2):
            if(len(cur_cat) != 0):
              pattern = re.compile(r'(^\s*\d*\.?\s*' + cur_cat + r':?)')
              match = pattern.search(acum)
              if match:
                acum = acum[:match.start()] + acum[match.end():]
              result[filename][cur_cat] = acum.strip()
              acum = ''
            cur_cat = category
        acum += text2
      if(len(cur_cat) > 0):
        pattern = re.compile(r'(^\s*\d*\.?\s*' + cur_cat + r':?)')
        match = pattern.search(acum)
        if match:
          acum = acum[:match.start()] + acum[match.end():]
        result[filename][cur_cat] = acum.strip()
  return result

In [ ]:
def cosineSimilarityBetweenTexts(text1, text2, remove_stopwords = False, tfidf = False):
  text1 = text1.lower()
  text2 = text2.lower()

  if (remove_stopwords):
    text1_tokens = word_tokenize(text1)
    text1_filtered_arr = [word for word in text1_tokens if word not in stopwords.words('english')]
    text1 = " ".join(text1_filtered_arr)

    text2_tokens = word_tokenize(text2)
    text2_filtered_arr = [word for word in text2_tokens if word not in stopwords.words('english')]
    text2 = " ".join(text2_filtered_arr)

  data = [text1, text2]

  vectorizer = None
  if tfidf:
    vectorizer = CountVectorizer()
  else:
    vectorizer = TfidfVectorizer()

  vector_matrix = vectorizer.fit_transform(data)
  cosine_similarity_matrix = cosine_similarity(vector_matrix)
  return cosine_similarity_matrix[0,1]

In [ ]:
def getDfFromResultOrderedByDate(result):
  df = pd.DataFrame(result)
  df = df.transpose()
  df = df.sort_values(by=['year', 'month'], ascending=[True, True])
  return df

In [ ]:
#Lista com as matrizes de similaridade dos produtos por categoria
def getListOfSimilarityMatrixesPerCategory(df, categories, remove_stopwords, tfidf):
  similarities = []
  for category in categories:
    allIndexes = df.index
    allTexts = df.loc[:,category]
    similarity = np.zeros((len(allIndexes),len(allIndexes)))
    for i in range(len(allIndexes)):
      for j in range(len(allIndexes)):
        similarity[i,j] = cosineSimilarityBetweenTexts(allTexts[i], allTexts[j], remove_stopwords = remove_stopwords, tfidf = tfidf)
    similarities.append(similarity)
  return similarities

In [ ]:
def getDfFromSimilaritiesMean(similaritiesMean, allIndexes):
  similarityDf = pd.DataFrame(similaritiesMean)
  similarityDf.columns = allIndexes
  similarityDf.index = allIndexes
  styledDf = similarityDf.style.background_gradient(cmap='viridis')
  return styledDf

## Apple

Foi notado um padrão de mudança de padrões de sustentabilidade, ou menos dos relatórios:

Celulares lançados entre 2009 e 2017 têm aspectos sustentáveis similares entre si

Celulares lançados entre 2019 e 2022 têm aspectos sustentáveis similares entre si

Celulares lançados em 2023 têm aspector sustentáveis similares entre si


In [ ]:
categories = ['Energy Efficiency', 'Recycled and Renewable Content', 'Longevity and Durability', 'Emission Reduction Initiatives',
              'Material Use', 'Packaging', 'Ethical Sourcing and Supplier Code of Conduct']

In [ ]:
result = parseGeneratedTextCategories('generated_text_apple', categories)
df = getDfFromResultOrderedByDate(result)
df

,month,year,Energy Efficiency,Recycled and Renewable Content,Longevity and Durability,Emission Reduction Initiatives,Material Use,Packaging,Ethical Sourcing and Supplier Code of Conduct
iphone-3g-060809-res.txt,06,09,"The iPhone 3G is designed with energy efficiency in mind. According to the report, the device uses power-efficient components and software that intelligently manage power consumption. The Apple USB power adapter outperforms the stringent requirements of the ENERGY STAR specification for external power supplies. The adapter's efficiency varies depending on the voltage and power output, with a maximum efficiency of 70% at 115V and 75% at 230V. The report also states that the battery chemistry is lithium-ion polymer, which is a more efficient and environmentally friendly alternative to traditional batteries.","The iPhone 3G packaging is almost entirely recyclable, and the retail box is made primarily from bio-based materials, including fiberboard containing 90% post-consumer recycled content. The report also states that iPhone 3G uses post-consumer recycled materials in its design, but does not provide specific data on the amount of recycled content used.",The report does not provide specific data on the longevity and durability of the iPhone 3G.,"The iPhone 3G is designed to reduce greenhouse gas emissions. According to the report, the device emits 55 kg CO2e over its life cycle, which is a reduction of 37% compared to the iPhone 3G (85g CO2e/unit). The report also states that Apple seeks to minimize greenhouse gas emissions by setting stringent design-related goals for material and energy efficiency.","The report provides detailed information on the materials used in the iPhone 3G. The device contains 19g of plastic, 12.5g of display, 30g of stainless steel, and 4g of others. The report also states that the device's circuit boards are made of 19.5g of material.","The report provides detailed information on the packaging of the iPhone 3G. The packaging is almost entirely recyclable, and the retail box is made primarily from bio-based materials, including fiberboard containing 90% post-consumer recycled content. The report also states that the packaging is extremely material efficient, allowing more units to be transported in a single shipping container.",The report does not provide information on ethical sourcing and the supplier code of conduct for the iPhone 3G.
iphone-3gs-060809-res.txt,06,09,"The iPhone 3GS is designed to be energy efficient, with a power-efficient design and software that intelligently manages power consumption. According to the report, the Apple USB power adapter outperforms the stringent requirements of the ENERGY STAR specification for external power supplies. The report also provides data on the estimated greenhouse gas emissions for the iPhone 3GS over its life cycle, which is 55 kg CO2e.","The iPhone 3GS packaging is almost entirely recyclable, and the retail box is made primarily from bio-based materials, including fiberboard containing 90 percent post-consumer recycled content. Additionally, the report states that iPhone 3GS goes even further than the requirements of the RoHS Directive by incorporating more aggressive restrictions, such as arsenic-free glass, mercury-free LCD display, and free of BFRs and PVC.","The report does not provide specific data on the longevity and durability of the iPhone 3GS, but it does mention that the ultra-compact product and packaging designs lead the industry in material efficiency, which helps maximize shipping efficiency and reduce energy consumed during production.","The report states that Apple has taken the lead in restricting harmful substances from its products and packaging, and all Apple products comply with the strict European Directive on the Restriction of the Use of Certain Hazardous Substances in Electrical and Electronic Equipment (RoHS Directive). The report also mentions that Apple offers and participates in various product take-back a

In [ ]:
similaritiesList = getListOfSimilarityMatrixesPerCategory(df, categories, remove_stopwords = True, tfidf = True)
similaritiesMean = np.mean(similaritiesList, axis=0)
allIndexes = df.index
getDfFromSimilaritiesMean(similaritiesMean, allIndexes)

,iphone-3g-060809-res.txt,iphone-3gs-060809-res.txt,iphone-4-062410-res.txt,iphone-5-091212-res.txt,iphone-5s-091013-res.txt,iphone-6-090914-res.txt,iphone-5c-090914-res.txt,iphone-6splus-090915-res.txt,iphone-6s-090915-res.txt,iphone-7-090716-res.txt,iphone-8plus-091217-res.txt,iphone-x-091217-res.txt,iphone-8-091217-res.txt,iphone-11pro-091019-res.txt,iphone-11promax-091019-res.txt,iphone-11-091019-res.txt,iphone-se2-041520-res.txt,iphone-12promax-101320-res.txt,iphone-12pro-101320-res.txt,iphone-12mini-101320-res.txt,iphone-12-101320-res.txt,iphone-13mini-091421-res.txt,iphone-13-091421-res.txt,iphone-13pro-091421-res.txt,iphone-13promax-091421-res.txt,iphone-se3-031822-res.txt,iphone-14pro-090722-res.txt,iphone-14andplus-090722-res.txt,iphone-14promax-090722-res.txt,iphone-15andplus-091223-res.txt,iphone-15proandmax-091223-res.txt
iphone-3g-060809-res.txt,1.000000,0.492874,0.413494,0.514046,0.477188,0.415011,0.388939,0.363139,0.307553,0.303505,0.413542,0.430107,0.292045,0.214488,0.188357,0.212798,0.186759,0.227176,0.263826,0.237595,0.183689,0.179603,0.233931,0.195597,0.151449,0.172781,0.230000,0.175493,0.169212,0.187344,0.174639
iphone-3gs-060809-res.txt,0.492874,1.000000,0.457034,0.412676,0.376553,0.433859,0.376500,0.406849,0.447279,0.428477,0.470470,0.323425,0.252578,0.214532,0.176834,0.164059,0.204225,0.226403,0.230971,0.224959,0.201401,0.175075,0.226772,0.232924,0.171943,0.214574,0.220442,0.211081,0.168988,0.171057,0.216237
iphone-4-062410-res.txt,0.413494,0.457034,1.000000,0.419655,0.432222,0.460456,0.408168,0.387375,0.379863,0.457353,0.402018,0.390165,0.337981,0.291667,0.197236,0.250736,0.216221,0.263981,0.303800,0.247480,0.238135,0.202107,0.282593,0.235434,0.170271,0.190840,0.228028,0.192951,0.198404,0.173626,0.209582
iphone-5-091212-res.txt,0.514046,0.412676,0.419655,1.000000,0.535114,0.466320,0.456269,0.517608,0.413043,0.385349,0.473604,0.483590,0.312901,0.249658,0.219430,0.251351,0.201793,0.242802,0.265620,0.230582,0.206372,0.173550,0.236408,0.229183,0.166058,0.183423,0.264044,0.200837,0.163563,0.168414,0.184579
iphone-5s-091013-res.txt,0.477188,0.376553,0.432222,0.535114,1.000000,0.520302,0.541102,0.534020,0.369921,0.407690,0.441899,0.462161,0.366074,0.245155,0.234341,0.257316,0.196129,0.262340,0.298937,0.227321,0.197826,0.186784,0.230891,0.234021,0.150479,0.195462,0.237079,0.199122,0.129502,0.165700,0.205072
iphone-6-090914-res.txt,0.415011,0.433859,0.460456,0.466320,0.520302,1.000000,0.453975,0.493475,0.341037,0.375502,0.445919,0.455812,0.334361,0.256799,0.227137,0.242938,0.233905,0.286609,0.301542,0.245927,0.248769,0.211501,0.261159,0.270344,0.189860,0.238953,0.288371,0.286710,0.175599,0.242194,0.254767
iphone-5c-090914-res.txt,0.388939,0.376500,0.408168,0.456269,0.541102,0.453975,1.000000,0.448512,0.363994,0.363265,0.402803,0.432136,0.315907,0.246793,0.220034,0.266905,0.243405,0.243793,0.290504,0.242221,0.205569,0.199459,0.258139,0.220083,0.173474,0.227099,0.217627,0.201494,0.158708,0.169255,0.216130
iphone-6splus-090915-res.txt,0.363139,0.406849,0.387375,0.517608,0.534020,0.493475,0.448512,1.000000,0.495272,0.441738,0.518393,0.464203,0.323249,0.304928,0.210802,0.284065,0.272001,0.232525,0.294359,0.254157,0.214794,0.173401,0.290779,0.234053,0.161522,0.205727,0.258062,0.229518,0.160955,0.162254,0.230442
iphone-6s-090915-res.txt,0.307553,0.447279,0.379863,0.413043,0.369921,0.341037,0.363994,0.495272,1.000000,0.505889,0.343138,0.338035,0.222421,0.206292,0.166433,0.188208,0.173968,0.212134,0.204581,0.153325,0.150916,0.141760,0.185783,0.181788,0.127516,0.165268,0.145140,0.177957,0.126419,0.146229,0.189515
iphone-7-090716-res.txt,0.303505,0.428477,0.457353,0.385349,0.407690,0.375502,0.363265,0.441738,0.505889,1.000000,0.445048,0.490493,0.447260,0.274225,0.225431,0.239671,0.250873,0.278118,0.294024,0.270549,0.270451,0.219175,0.285866,0.271664,0.208875,0.228481,0.225720,0.224931,0.220025,0.166407,0.235779


In [ ]:
similaritiesList = getListOfSimilarityMatrixesPerCategory(df, categories, remove_stopwords = True, tfidf = False)
similaritiesMean = np.mean(similaritiesList, axis=0)
allIndexes = df.index
getDfFromSimilaritiesMean(similaritiesMean, allIndexes)

,iphone-3g-060809-res.txt,iphone-3gs-060809-res.txt,iphone-4-062410-res.txt,iphone-5-091212-res.txt,iphone-5s-091013-res.txt,iphone-6-090914-res.txt,iphone-5c-090914-res.txt,iphone-6splus-090915-res.txt,iphone-6s-090915-res.txt,iphone-7-090716-res.txt,iphone-8plus-091217-res.txt,iphone-x-091217-res.txt,iphone-8-091217-res.txt,iphone-11pro-091019-res.txt,iphone-11promax-091019-res.txt,iphone-11-091019-res.txt,iphone-se2-041520-res.txt,iphone-12promax-101320-res.txt,iphone-12pro-101320-res.txt,iphone-12mini-101320-res.txt,iphone-12-101320-res.txt,iphone-13mini-091421-res.txt,iphone-13-091421-res.txt,iphone-13pro-091421-res.txt,iphone-13promax-091421-res.txt,iphone-se3-031822-res.txt,iphone-14pro-090722-res.txt,iphone-14andplus-090722-res.txt,iphone-14promax-090722-res.txt,iphone-15andplus-091223-res.txt,iphone-15proandmax-091223-res.txt
iphone-3g-060809-res.txt,1.000000,0.361322,0.302263,0.384097,0.359846,0.285622,0.276969,0.248823,0.218944,0.204035,0.277580,0.298477,0.199504,0.127580,0.111319,0.127081,0.112207,0.134913,0.164075,0.146527,0.107788,0.105616,0.140683,0.114618,0.087816,0.098813,0.143554,0.100526,0.098167,0.108484,0.098861
iphone-3gs-060809-res.txt,0.361322,1.000000,0.339597,0.281822,0.257322,0.310674,0.273282,0.287486,0.314102,0.291873,0.321235,0.212115,0.168908,0.130605,0.103481,0.096169,0.121581,0.142793,0.148510,0.141046,0.124395,0.103832,0.141120,0.146231,0.098565,0.130827,0.137011,0.129553,0.102606,0.098375,0.126552
iphone-4-062410-res.txt,0.302263,0.339597,1.000000,0.297980,0.312844,0.330538,0.287758,0.259886,0.259708,0.320395,0.270983,0.260933,0.232973,0.182774,0.117538,0.154400,0.131888,0.167661,0.194765,0.155938,0.148029,0.122274,0.180385,0.148426,0.099525,0.116198,0.145459,0.115237,0.122073,0.099638,0.124686
iphone-5-091212-res.txt,0.384097,0.281822,0.297980,1.000000,0.403407,0.326193,0.326811,0.375807,0.308533,0.258720,0.339058,0.345126,0.215128,0.150943,0.132024,0.153674,0.120673,0.148696,0.165407,0.142799,0.121906,0.100094,0.142515,0.137739,0.094723,0.106950,0.167139,0.118082,0.094886,0.095925,0.105522
iphone-5s-091013-res.txt,0.359846,0.257322,0.312844,0.403407,1.000000,0.377270,0.417676,0.401768,0.261610,0.281532,0.298637,0.325217,0.255271,0.145356,0.139963,0.153710,0.114392,0.160667,0.188507,0.141764,0.116284,0.108439,0.138167,0.141283,0.085058,0.114518,0.149095,0.115777,0.075064,0.093429,0.118260
iphone-6-090914-res.txt,0.285622,0.310674,0.330538,0.326193,0.377270,1.000000,0.325526,0.352854,0.226019,0.258314,0.302400,0.312732,0.225722,0.157442,0.134050,0.143981,0.139653,0.180873,0.194179,0.158132,0.155931,0.126546,0.162960,0.172947,0.112714,0.146682,0.188815,0.180045,0.108129,0.143280,0.154246
iphone-5c-090914-res.txt,0.276969,0.273282,0.287758,0.326811,0.417676,0.325526,1.000000,0.319936,0.249308,0.253706,0.263705,0.296049,0.214037,0.149339,0.128888,0.162974,0.146631,0.152800,0.186449,0.154622,0.128116,0.116654,0.161906,0.137510,0.100621,0.142062,0.136828,0.124387,0.095541,0.097500,0.126273
iphone-6splus-090915-res.txt,0.248823,0.287486,0.259886,0.375807,0.401768,0.352854,0.319936,1.000000,0.368886,0.316048,0.368708,0.321480,0.216974,0.190505,0.123062,0.175642,0.167835,0.143165,0.188972,0.160505,0.130525,0.100421,0.183567,0.144376,0.092483,0.122442,0.162226,0.143350,0.095812,0.093399,0.137409
iphone-6s-090915-res.txt,0.218944,0.314102,0.259708,0.308533,0.261610,0.226019,0.249308,0.368886,1.000000,0.372478,0.225367,0.228985,0.142092,0.122612,0.096376,0.114964,0.102516,0.129989,0.122880,0.088447,0.087834,0.082581,0.111806,0.107916,0.072281,0.094474,0.084931,0.105261,0.073389,0.082920,0.109701
iphone-7-090716-res.txt,0.204035,0.291873,0.320395,0.258720,0.281532,0.258314,0.253706,0.316048,0.372478,1.000000,0.308802,0.363969,0.340759,0.174005,0.140679,0.154442,0.161031,0.181344,0.192825,0.173399,0.176897,0.135387,0.188222,0.180559,0.125282,0.143064,0.142544,0.140226,0.136997,0.097021,0.141265


## Google

Similiar a Apple também há um padrão mas ele é mais fraco até 2020.

De 2021 a 2023 os relatórios aparentam ser mais similares.

In [ ]:
categories = ['Energy Efficiency', 'Recycled and Renewable Content', 'Longevity and Durability', 'Emission Reduction Initiatives',
              'Material Use', 'Packaging', 'Ethical Sourcing and Supplier Code of Conduct']

In [ ]:
result = parseGeneratedTextCategories('generated_text_google', categories)
df = getDfFromResultOrderedByDate(result)
df

,month,year,Energy Efficiency,Recycled and Renewable Content,Longevity and Durability,Emission Reduction Initiatives,Material Use,Packaging,Ethical Sourcing and Supplier Code of Conduct
pixel-2-100417-res.txt,10,17,"The Google Pixel 2 has a power adapter with a Level VI efficiency rating, which means it uses less energy than other adapters on the market. The phone also has a power-management software that maximizes battery-charging efficiency and extends battery life during use. According to the text, the estimated GHG emissions for the Pixel 2 are 66.4 kg CO2e over its three-year life cycle, which is a relatively low amount compared to other electronic products.","The packaging for the Pixel 2 uses 99% paper and fiber-based materials, and the chipboard material that forms the underlying structural layer of the box base and lid is made of 100% recycled content. Additionally, the phone is designed to be light and compact, which minimizes the amount of materials used during production.","The Pixel 2 has a battery that is designed to last for up to 30 months on standby mode, which is a relatively long time compared to other electronic devices. The phone also has a compact design that allows it to be easily transported and stored, which can help to reduce waste and extend its lifespan.","Google has implemented various initiatives to reduce the environmental impact of its products, including using materials that are restricted substances, such as lead, mercury, and cadmium, and designing products that are energy efficient. The company has also set goals to reduce its greenhouse gas emissions and water usage.","The Pixel 2 is designed to use as few materials as possible during production, which can help to reduce waste and conserve natural resources. The phone has a total of 143 g of materials, which is a relatively low amount compared to other electronic devices. The most commonly used materials in the Pixel 2 are aluminum, battery, and display assembly.","The packaging for the Pixel 2 is designed to be light and compact, which minimizes the amount of materials used during production. The packaging also uses 99% paper and fiber-based materials, which are renewable and can be recycled.","Google has established a Supplier Code of Conduct that sets out the company's expectations for its manufacturing partners, including that they must comply with all applicable laws and regulations regarding working conditions, health and safety, and the environment. The company also conducts regular audits to ensure compliance with these expectations."
pixel-2xl-100417-res.txt,10,17,"The Google Pixel 2 XL has a Level VI efficiency rating for its power adapter, which means it is designed to use less energy. In addition, the phone's display uses a pOLED display that is mercury-free, arsenic-free, and BFR-free. The device also has a power-saving mode that can extend battery life. According to the text, the estimated GHG emissions for the Pixel 2 XL are 71.5 kg CO2e over its three-year life cycle.","The packaging for the Pixel 2 XL is made of 99% paper and fiber-based materials, and the chipboard material used in the box base and lid is made of 100% recycled content.","The Pixel 2 XL is designed to be light and compact, which allows materials to be used more efficiently and minimizes the amount of packaging. The device also has a battery that is designed to last for three years.","The Google Pixel 2 XL has been designed to reduce its environmental impact throughout its life cycle. This includes using materials that are restricted substances, such as lead, mercury, cadmium, and brominated flame retardants, and using energy-efficient components, such as a power adapter with a Level VI efficiency rating and a display that uses a pOLED display.","The Google Pixel 2 XL contains 175g of materials overall, with 46g of aluminum, 8g of battery, and 30g of other metals. The device also has a display assembly that weighs 13g and miscellaneous components that weigh 

In [ ]:
similaritiesList = getListOfSimilarityMatrixesPerCategory(df, categories, remove_stopwords = True, tfidf = True)
similaritiesMean = np.mean(similaritiesList, axis=0)
allIndexes = df.index
getDfFromSimilaritiesMean(similaritiesMean, allIndexes)

,pixel-2-100417-res.txt,pixel-2xl-100417-res.txt,pixel-3-100918-res.txt,pixel-3xl-100918-res.txt,pixel-3a-100918-res.txt,pixel-4-102419-res.txt,pixel-4xl-102419-res.txt,pixel-5-102020-res.txt,pixel-4a-5g-102020-res.txt,pixel-4a-102020-res.txt,pixel-5a-with-5g-082021-res.txt,pixel-6-102021-res.txt,pixel-6a-072022-res.txt,pixel-7-102022-res.txt,pixel-7-pro-102022-res.txt,pixel-7a-052023-res.txt,pixel-fold-062023-res.txt,pixel-8-pro-102023-res.txt,pixel-8-102023-res.txt
pixel-2-100417-res.txt,1.000000,0.430962,0.397323,0.400898,0.399222,0.410044,0.368070,0.414553,0.350840,0.426969,0.340928,0.437923,0.298771,0.388046,0.326060,0.294691,0.386018,0.374068,0.365932
pixel-2xl-100417-res.txt,0.430962,1.000000,0.454785,0.474892,0.478990,0.435743,0.456365,0.397611,0.353005,0.452554,0.384288,0.460890,0.404956,0.476439,0.396106,0.397322,0.438923,0.452136,0.414246
pixel-3-100918-res.txt,0.397323,0.454785,1.000000,0.423741,0.597810,0.533435,0.463219,0.437613,0.397225,0.494394,0.524987,0.464439,0.443802,0.471884,0.446211,0.330938,0.474225,0.382209,0.552370
pixel-3xl-100918-res.txt,0.400898,0.474892,0.423741,1.000000,0.437874,0.397460,0.484496,0.412318,0.355724,0.451686,0.353601,0.430438,0.362885,0.398431,0.392790,0.375681,0.362522,0.479409,0.424044
pixel-3a-100918-res.txt,0.399222,0.478990,0.597810,0.437874,1.000000,0.528896,0.531938,0.486773,0.450607,0.480955,0.518627,0.548213,0.549506,0.599843,0.584193,0.368027,0.483927,0.427140,0.556396
pixel-4-102419-res.txt,0.410044,0.435743,0.533435,0.397460,0.528896,1.000000,0.531668,0.558864,0.540364,0.534120,0.558827,0.490173,0.486314,0.544904,0.456069,0.409203,0.455477,0.472290,0.482594
pixel-4xl-102419-res.txt,0.368070,0.456365,0.463219,0.484496,0.531938,0.531668,1.000000,0.520821,0.448869,0.497394,0.474666,0.408344,0.444777,0.517305,0.448319,0.381914,0.472112,0.391100,0.534813
pixel-5-102020-res.txt,0.414553,0.397611,0.437613,0.412318,0.486773,0.558864,0.520821,1.000000,0.512843,0.563285,0.533405,0.550994,0.453657,0.496887,0.494492,0.372406,0.444413,0.474556,0.441705
pixel-4a-5g-102020-res.txt,0.350840,0.353005,0.397225,0.355724,0.450607,0.540364,0.448869,0.512843,1.000000,0.616253,0.494522,0.478234,0.529948,0.548484,0.481067,0.385884,0.454656,0.427115,0.477516
pixel-4a-102020-res.txt,0.426969,0.452554,0.494394,0.451686,0.480955,0.534120,0.497394,0.563285,0.616253,1.000000,0.575497,0.550250,0.492008,0.539494,0.495102,0.425022,0.478080,0.472243,0.484618


In [ ]:
similaritiesList = getListOfSimilarityMatrixesPerCategory(df, categories, remove_stopwords = True, tfidf = False)
similaritiesMean = np.mean(similaritiesList, axis=0)
allIndexes = df.index
getDfFromSimilaritiesMean(similaritiesMean, allIndexes)

,pixel-2-100417-res.txt,pixel-2xl-100417-res.txt,pixel-3-100918-res.txt,pixel-3xl-100918-res.txt,pixel-3a-100918-res.txt,pixel-4-102419-res.txt,pixel-4xl-102419-res.txt,pixel-5-102020-res.txt,pixel-4a-5g-102020-res.txt,pixel-4a-102020-res.txt,pixel-5a-with-5g-082021-res.txt,pixel-6-102021-res.txt,pixel-6a-072022-res.txt,pixel-7-102022-res.txt,pixel-7-pro-102022-res.txt,pixel-7a-052023-res.txt,pixel-fold-062023-res.txt,pixel-8-pro-102023-res.txt,pixel-8-102023-res.txt
pixel-2-100417-res.txt,1.000000,0.300787,0.270667,0.267173,0.269724,0.279291,0.245916,0.273915,0.230361,0.285838,0.231519,0.301755,0.193918,0.263714,0.206364,0.190097,0.255888,0.250892,0.243953
pixel-2xl-100417-res.txt,0.300787,1.000000,0.332104,0.347194,0.357355,0.323338,0.336423,0.283790,0.243967,0.325739,0.265577,0.337952,0.295569,0.356042,0.289212,0.282771,0.308176,0.328938,0.292161
pixel-3-100918-res.txt,0.270667,0.332104,1.000000,0.302036,0.464654,0.397333,0.346184,0.314068,0.282391,0.366848,0.387294,0.338856,0.321231,0.334327,0.320250,0.233649,0.347671,0.265476,0.409728
pixel-3xl-100918-res.txt,0.267173,0.347194,0.302036,1.000000,0.312994,0.287153,0.353593,0.286878,0.251431,0.312092,0.240845,0.302925,0.257194,0.284052,0.272914,0.266661,0.247127,0.343528,0.296545
pixel-3a-100918-res.txt,0.269724,0.357355,0.464654,0.312994,1.000000,0.396256,0.424232,0.360744,0.348547,0.351288,0.394307,0.412798,0.420740,0.466468,0.448681,0.269758,0.352286,0.303859,0.428507
pixel-4-102419-res.txt,0.279291,0.323338,0.397333,0.287153,0.396256,1.000000,0.417148,0.421195,0.428536,0.393743,0.420848,0.354453,0.356980,0.403715,0.329631,0.303444,0.329267,0.348539,0.354940
pixel-4xl-102419-res.txt,0.245916,0.336423,0.346184,0.353593,0.424232,0.417148,1.000000,0.398089,0.338050,0.383815,0.352771,0.302695,0.327892,0.388474,0.330539,0.272808,0.335390,0.286496,0.408417
pixel-5-102020-res.txt,0.273915,0.283790,0.314068,0.286878,0.360744,0.421195,0.398089,1.000000,0.394207,0.443608,0.392452,0.425331,0.324549,0.353783,0.355535,0.265894,0.314214,0.354239,0.307523
pixel-4a-5g-102020-res.txt,0.230361,0.243967,0.282391,0.251431,0.348547,0.428536,0.338050,0.394207,1.000000,0.474449,0.382243,0.348102,0.422774,0.413546,0.348665,0.306846,0.328650,0.313336,0.350991
pixel-4a-102020-res.txt,0.285838,0.325739,0.366848,0.312092,0.351288,0.393743,0.383815,0.443608,0.474449,1.000000,0.439412,0.421705,0.359846,0.394983,0.355919,0.309214,0.340428,0.336339,0.338449


## Samsung

Devido à falta de informação disponível não foi possível fazer análises significativas

In [ ]:
categories = ['Energy Efficiency', 'Recycled and Renewable Content', 'Longevity and Durability', 'Emission Reduction Initiatives',
              'Material Use', 'Packaging', 'Ethical Sourcing and Supplier Code of Conduct']

In [ ]:
result = parseGeneratedTextCategories('generated_text_samsung', categories)
df = getDfFromResultOrderedByDate(result)
df

,month,year,Energy Efficiency,Recycled and Renewable Content,Longevity and Durability,Emission Reduction Initiatives,Material Use,Packaging,Ethical Sourcing and Supplier Code of Conduct
galaxy-s23-020123-res.txt,02,23,"Samsung is working to reduce energy consumption throughout the product life cycle, from raw material sourcing to distribution and use. They are expanding the use of renewable energy at their business sites around the world and have established reduction plans for the use of potentially hazardous substances.","Samsung is using recycled materials and collecting e-waste to extract materials for reuse. They are also using renewable energy at their business sites around the world. By 2030, they aim to have 50% of the plastic used in their DX products to incorporate recycled resin, and by 2050, they aim to have 100%.",Samsung is designing products with longevity and durability in mind. They are using environmentally-friendly materials and reducing waste in their production processes. They also provide repair and reuse programs for customers.,Samsung is working to reduce greenhouse gas emissions throughout the product life cycle. They are implementing energy-saving technologies and reducing packaging waste. They also aim to eliminate all single-use plastics in mobile product packaging by 2025.,Samsung is reducing the amount of material used in their products and packaging. They are using recycled paper for packaging and reducing the weight and volume of packaging materials. They also aim to eliminate all single-use plastics in mobile product packaging by 2025.,Samsung is reducing the environmental impact of their product packaging. They are replacing plastic packaging and protective films with paper and recycled materials to the extent possible. They are also reducing the volume and weight of packaging to mitigate greenhouse gas emissions in the transportation and shipping process.,"Samsung is working to ensure that their suppliers have robust occupational health and safety management systems in place. They are encouraging their suppliers to attain certification for international standards and reflect related outcomes in the comprehensive supplier evaluations. As of 2021, all business sites of Samsung Electronics have attained the certifications, and 86% of suppliers subject to comprehensive evaluation of partner have acquired the certification."
galaxy-s23plus-020123-res.txt,02,23,"Samsung has set a goal to achieve zero deforestation in its supply chain by 2025 and is working to reduce energy consumption across its operations. The company has implemented various energy-saving measures, such as using LED lighting and energy-efficient HVAC systems, and has reduced its energy consumption by 20% since 2015.","Samsung has committed to using 50% recycled materials in its products by 2030 and is working to increase its use of renewable energy. The company has implemented various initiatives to reduce waste and increase recycling, such as using recycled plastic in its packaging and implementing a recycling program for used mobile phones.","Samsung has designed its products to be more durable and long-lasting, with a focus on reducing e-waste. The company has implemented various initiatives to extend the lifespan of its products, such as offering repair and repair services and providing software updates to extend the life of its products.","Samsung has set a goal to reduce its greenhouse gas emissions by 50% by 2030 and is working to implement various initiatives to achieve this goal. The company has implemented various measures to reduce emissions, such as using renewable energy, reducing energy consumption, and implementing carbon offsetting programs.","Samsung has committed to reducing the amount of material used in its products and is working to increase the use of recycled materials. The company has implemented various initiatives to reduce material usage, such as using minimalist design and reducing the amount of packagi

In [ ]:
similaritiesList = getListOfSimilarityMatrixesPerCategory(df, categories, remove_stopwords = True, tfidf = True)
similaritiesMean = np.mean(similaritiesList, axis=0)
allIndexes = df.index
getDfFromSimilaritiesMean(similaritiesMean, allIndexes)

,galaxy-s23-020123-res.txt,galaxy-s23plus-020123-res.txt,galaxy-s23ultra-020123-res.txt,galaxy-zfold5-072623-res.txt,galaxy-zflip-072623-res.txt
galaxy-s23-020123-res.txt,1.000000,0.399338,0.416293,0.460024,0.424731
galaxy-s23plus-020123-res.txt,0.399338,1.000000,0.424101,0.447067,0.539544
galaxy-s23ultra-020123-res.txt,0.416293,0.424101,1.000000,0.384251,0.451144
galaxy-zfold5-072623-res.txt,0.460024,0.447067,0.384251,1.000000,0.435831
galaxy-zflip-072623-res.txt,0.424731,0.539544,0.451144,0.435831,1.000000


In [ ]:
similaritiesList = getListOfSimilarityMatrixesPerCategory(df, categories, remove_stopwords = True, tfidf = False)
similaritiesMean = np.mean(similaritiesList, axis=0)
allIndexes = df.index
getDfFromSimilaritiesMean(similaritiesMean, allIndexes)

,galaxy-s23-020123-res.txt,galaxy-s23plus-020123-res.txt,galaxy-s23ultra-020123-res.txt,galaxy-zfold5-072623-res.txt,galaxy-zflip-072623-res.txt
galaxy-s23-020123-res.txt,1.000000,0.266732,0.279202,0.318495,0.295915
galaxy-s23plus-020123-res.txt,0.266732,1.000000,0.295656,0.303717,0.392286
galaxy-s23ultra-020123-res.txt,0.279202,0.295656,1.000000,0.254346,0.319282
galaxy-zfold5-072623-res.txt,0.318495,0.303717,0.254346,1.000000,0.295577
galaxy-zflip-072623-res.txt,0.295915,0.392286,0.319282,0.295577,1.000000
